In [3]:
cd F:\MTP

F:\MTP


In [4]:
import os
import torch
from collections import defaultdict
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf

In [5]:
import os
path = os.listdir('EmoDB/wav')

We extract speaker embeddings(x-vectors) with a pretrained TDNN model using SpeechBrain from each utternace with a size of 512.

In [6]:
import torchaudio
from speechbrain.pretrained import EncoderClassifier
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-xvect-voxceleb", savedir="pretrained_models/spkrec-xvect-voxceleb")

Each utterance from the database is name according to the scheme :
    Positions 1-2: number of speaker
    Positions 3-5: code for text
    Position 6: emotion (sorry, letter stands for german emotion word)
    Position 7: if there are more than two versions these are numbered a, b, c....

Example: 03a01Fa.wav is the audio file from Speaker 03 speaking text a01 with
the emotion "Freude" (Happiness in German).

So now,we extract the emotion from the filenames and consider it as y in our model.

In [7]:
emotion_code = {
    'W':0, #anger
    'L':1, #boredom
    'E':2, #disgust
    'A':3, #fear
    'F':4, #happy
    'T':5, #sad
    'N':6  #neutral
}

speaker_code = {
    '03':0,
    '08':1,
    '09':2,
    '10':3,
    '11':4,
    '12':5,
    '13':6,
    '14':7,
    '15':8,
    '16':9
}

In [8]:
x_vec_size = 512
X = torch.zeros([len(path),x_vec_size])
y = torch.zeros([len(path),7],dtype = int)
for i in range(len(path)):
    signal, fs =torchaudio.load('EmoDB/wav/'+path[i])
    embeddings = classifier.encode_batch(signal)
    X[i] = embeddings[0][0]
    y[i][emotion_code[path[i][5]]]=1

In [9]:
# ys = torch.zeros([len(path),10],dtype = int)
# yg = torch.zeros([len(path),2],dtype = int)
# for i in range(len(path)):
#     ys[i][speaker_code[path[i][:2]]]=1
#     yg[i][gender_code[path[i][:2]]]=1

In [10]:
#print(X.shape,y.shape,ys.shape)

In [11]:
#print(ys[0])

Convert them into numpy arrays so that we can train them easily

In [12]:
X=X.detach().numpy()
y=y.detach().numpy()
#ys=ys.detach().numpy()

In [13]:
dict_X=defaultdict(list)
dict_y=defaultdict(list)
#dict_ys=defaultdict(list)
for i in range(len(path)):
    dict_X[path[i][:2]].append(X[i])
    dict_y[path[i][:2]].append(y[i])
   # dict_ys[path[i][:2]].append(ys[i])

In [14]:
#print(X.shape,ys.shape)

To make this evaluation speaker independent, we separate the data speaker-wise and use leave-one-out validation.

In [15]:
dict = {0:"03",1:"08",2:"09",3:"10",4:"11",5:"12",6:"13",7:"14",8:"15",9:"16"}

In [16]:
gender_code = {
    '03':0,
    '08':1,
    '09':1,
    '10':0,
    '11':0,
    '12':0,
    '13':1,
    '14':1,
    '15':0,
    '16':1
}

In [17]:
#requirements


from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics
from sklearn.metrics import accuracy_score
from keras.callbacks import ModelCheckpoint


Since the x-vectors are extracted before the last layer from a pretrained model, we just add softmax layer to get the output layer.

In [18]:
def train_and_test(X,y,X_test,y_test,X_valid,y_valid):
    model = Sequential()
    model.add(Dense(7, input_shape=(len(X[0]),)))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    checkpoint_filepath = './tmp/checkpoint'
    model_checkpoint_callback =ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_accuracy',mode='max',save_best_only=True)
    model.fit(X,y,batch_size=20, epochs=200,callbacks=model_checkpoint_callback,validation_data=(X_valid,y_valid))
    loss,acc=model.evaluate(X_test,y_test)
    print(acc)
    return acc

In [19]:
def train_and_test_speak(X,y,X_test,y_test,X_valid,y_valid):
    model = Sequential()
    model.add(Dense(10, input_shape=(len(X[0]),)))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    checkpoint_filepath = './tmp/checkpoint_speakers'
    model_checkpoint_callback =ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_accuracy',mode='max',save_best_only=True)
    model.fit(X,y,batch_size=20, epochs=200,callbacks=model_checkpoint_callback,validation_data=(X_valid,y_valid))
    loss,acc=model.evaluate(X_test,y_test)
    print(acc)
    return acc

In [20]:
temp = 9
acc = 0
accuracies = [0]*10
for i in range(10):
    if temp == 0:
        valid = 9
    else:
        valid = temp-1
    
    a = list((dict_X[dict[i]] for i in range(10) if i not in [temp,valid] ))
    b = list((dict_y[dict[i]] for i in range(10) if i not in [temp,valid]))
    X_ = np.concatenate(a)
    X_test = np.array(dict_X[dict[temp]])
    X_valid = np.array(dict_X[dict[valid]])
    y_test = np.array(dict_y[dict[temp]])
    y_valid = np.array(dict_y[dict[valid]])
    y_ = np.concatenate(b)
    temp -= 1
    print(9-temp)
    temp_acc = train_and_test(X_,y_,X_test,y_test,X_valid,y_valid)
    accuracies[temp]=temp_acc
    print(f'Accuracy for iteration: {temp_acc}')
    acc += temp_acc

1
Epoch 1/200
15/21 [====================>.........] - ETA: 0s - loss: 18.0489 - accuracy: 0.1900WARNING:tensorflow:From C:\Users\Pramodh\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\training\tracking\tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./tmp\checkpoint\assets
21/21 [==============================] - 2s 81ms/step - loss: 15.9101 - accuracy: 0.1985 - val_loss: 9.0016 - val_accuracy: 0.2500
Epoch 2/200
21/21 [==============================] - 1s 36ms/step - loss: 5.4626 - accuracy: 0.3627 - val_loss: 5.5915 - val_accuracy: 0.3571
Epoch 3/200
21/21 [==============================] - 1s 29ms/step - loss: 3

21/21 [==============================] - 0s 2ms/step - loss: 0.0121 - accuracy: 1.0000 - val_loss: 1.6061 - val_accuracy: 0.7143
Epoch 45/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0120 - accuracy: 1.0000 - val_loss: 1.7359 - val_accuracy: 0.7143
Epoch 46/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0124 - accuracy: 1.0000 - val_loss: 1.6193 - val_accuracy: 0.7321
Epoch 47/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0134 - accuracy: 1.0000 - val_loss: 1.5798 - val_accuracy: 0.7143
Epoch 48/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0120 - accuracy: 1.0000 - val_loss: 1.6857 - val_accuracy: 0.6964
Epoch 49/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0120 - accuracy: 1.0000 - val_loss: 1.6208 - val_accuracy: 0.7143
Epoch 50/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0122 - accuracy: 1.0000 - val_loss: 1.5226 - val_accuracy: 0.7321
Epoch 51/200
21/21 

Epoch 102/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 1.8653 - val_accuracy: 0.6964
Epoch 103/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 1.8610 - val_accuracy: 0.6964
Epoch 104/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 1.8308 - val_accuracy: 0.6964
Epoch 105/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0034 - accuracy: 1.0000 - val_loss: 1.8331 - val_accuracy: 0.6964
Epoch 106/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 1.9162 - val_accuracy: 0.6786
Epoch 107/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 1.8744 - val_accuracy: 0.6964
Epoch 108/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 1.8750 - val_accuracy: 0.6964

21/21 [==============================] - 0s 2ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 1.9622 - val_accuracy: 0.6964
Epoch 160/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 2.0055 - val_accuracy: 0.6786
Epoch 161/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 1.9270 - val_accuracy: 0.6964
Epoch 162/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 2.0355 - val_accuracy: 0.6964
Epoch 163/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 2.0114 - val_accuracy: 0.6964
Epoch 164/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 1.9444 - val_accuracy: 0.6786
Epoch 165/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 1.9879 - val_accuracy: 0.6964
Epoch 166/200

21/21 [==============================] - 1s 32ms/step - loss: 0.7124 - accuracy: 0.7951 - val_loss: 1.7179 - val_accuracy: 0.6667
Epoch 10/200
21/21 [==============================] - 0s 8ms/step - loss: 0.6701 - accuracy: 0.8000 - val_loss: 2.2005 - val_accuracy: 0.6087
Epoch 11/200
21/21 [==============================] - 0s 6ms/step - loss: 0.4367 - accuracy: 0.8512 - val_loss: 2.2208 - val_accuracy: 0.6087
Epoch 12/200
21/21 [==============================] - 0s 5ms/step - loss: 0.3846 - accuracy: 0.8707 - val_loss: 1.7579 - val_accuracy: 0.6667
Epoch 13/200
21/21 [==============================] - 0s 5ms/step - loss: 0.3346 - accuracy: 0.8780 - val_loss: 1.6496 - val_accuracy: 0.6522
Epoch 14/200
21/21 [==============================] - 0s 3ms/step - loss: 0.3488 - accuracy: 0.8805 - val_loss: 2.1363 - val_accuracy: 0.6522
Epoch 15/200
21/21 [==============================] - 0s 3ms/step - loss: 0.4349 - accuracy: 0.8366 - val_loss: 2.0466 - val_accuracy: 0.6232
Epoch 16/200
21/21

...

22/22 [==============================] - 0s 4ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.9936 - val_accuracy: 0.8163
Epoch 180/200
22/22 [==============================] - 0s 2ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.9814 - val_accuracy: 0.7959
Epoch 181/200
22/22 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.9648 - val_accuracy: 0.7959
Epoch 182/200
22/22 [==============================] - 0s 3ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 1.0296 - val_accuracy: 0.8163
Epoch 183/200
22/22 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 1.0031 - val_accuracy: 0.8163
Epoch 184/200
22/22 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 1.0145 - val_accuracy: 0.8163
Epoch 185/200
22/22 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.9918 - val_accuracy: 0.8163
Epoch 186/200

Epoch 24/200
21/21 [==============================] - 0s 3ms/step - loss: 0.1232 - accuracy: 0.9639 - val_loss: 0.9697 - val_accuracy: 0.7465
Epoch 25/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0839 - accuracy: 0.9735 - val_loss: 0.9669 - val_accuracy: 0.7465
Epoch 26/200
21/21 [==============================] - 0s 3ms/step - loss: 0.1706 - accuracy: 0.9325 - val_loss: 1.4991 - val_accuracy: 0.7465
Epoch 27/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0918 - accuracy: 0.9759 - val_loss: 1.2121 - val_accuracy: 0.6901
Epoch 28/200
21/21 [==============================] - 0s 3ms/step - loss: 0.0972 - accuracy: 0.9590 - val_loss: 1.5396 - val_accuracy: 0.6620
Epoch 29/200
21/21 [==============================] - 1s 27ms/step - loss: 0.0702 - accuracy: 0.9783 - val_loss: 0.7550 - val_accuracy: 0.8028
Epoch 30/200
21/21 [==============================] - 1s 29ms/step - loss: 0.0725 - accuracy: 0.9735 - val_loss: 0.8659 - val_accuracy: 0.8169
Epoc

Epoch 79/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 0.8021 - val_accuracy: 0.8310
Epoch 80/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 0.8554 - val_accuracy: 0.8028
Epoch 81/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0052 - accuracy: 1.0000 - val_loss: 0.7735 - val_accuracy: 0.8310
Epoch 82/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0048 - accuracy: 1.0000 - val_loss: 0.7379 - val_accuracy: 0.8310
Epoch 83/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0048 - accuracy: 1.0000 - val_loss: 0.7886 - val_accuracy: 0.8310
Epoch 84/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0050 - accuracy: 1.0000 - val_loss: 0.7586 - val_accuracy: 0.8310
Epoch 85/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0046 - accuracy: 1.0000 - val_loss: 0.7686 - val_accuracy: 0.8310
Epoch 

21/21 [==============================] - 0s 2ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.7523 - val_accuracy: 0.8310
Epoch 137/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.7357 - val_accuracy: 0.8310
Epoch 138/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.7503 - val_accuracy: 0.8310
Epoch 139/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.7040 - val_accuracy: 0.8310
Epoch 140/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.7554 - val_accuracy: 0.8310
Epoch 141/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.6818 - val_accuracy: 0.8310
Epoch 142/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.7587 - val_accuracy: 0.8310
Epoch 143/200

21/21 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.7115 - val_accuracy: 0.8451
Epoch 194/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.6748 - val_accuracy: 0.8451
Epoch 195/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.7176 - val_accuracy: 0.8451
Epoch 196/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.7155 - val_accuracy: 0.8451
Epoch 197/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.6799 - val_accuracy: 0.8451
Epoch 198/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.7187 - val_accuracy: 0.8451
Epoch 199/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.7201 - val_accuracy: 0.8451
Epoch 200/200

leave-one-speaker-out validation accuracies

In [21]:
accuracies

[0.517241358757019,
 0.6279069781303406,
 0.7105262875556946,
 0.7090908885002136,
 0.6857143044471741,
 0.7213114500045776,
 0.7101449370384216,
 0.7321428656578064,
 0.9014084339141846,
 0.8775510191917419]

In [22]:
print("Overall accuracy :"+ str(acc*10)+"%" )

Overall accuracy :71.93038523197174%


In [23]:
#model for speaker recognition
# Xtrain1, Xtest, ytrain1, ytest = train_test_split(X.numpy(), ys.numpy(), test_size=0.3, random_state=42)
# Xtrain, Xvalid, ytrain, yvalid = train_test_split(Xtrain1, ytrain1, test_size=0.15, random_state=42)
# model = Sequential()
# model.add(Dense(10, input_shape=(len(X[0]),)))
# model.add(Activation('softmax'))
# model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
# checkpoint_filepath = './tmp/checkpoint_speakers'
# model_checkpoint_callback =ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_accuracy',mode='max',save_best_only=True)
# model.fit(Xtrain,ytrain,batch_size=20, epochs=200,callbacks=model_checkpoint_callback,validation_data=(Xvalid,yvalid))
# loss,acc=model.evaluate(Xtest,ytest)
# print(acc)

In [24]:
# model for gender recognition
# Xtrain1, Xtest, ytrain1, ytest = train_test_split(X.numpy(), yg.numpy(), test_size=0.3, random_state=42)
# Xtrain, Xvalid, ytrain, yvalid = train_test_split(Xtrain1, ytrain1, test_size=0.15, random_state=42)
# model = Sequential()
# model.add(Dense(2, input_shape=(len(X[0]),)))
# model.add(Activation('softmax'))
# model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
# checkpoint_filepath = './tmp/checkpoint_speakers'
# model_checkpoint_callback =ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_accuracy',mode='max',save_best_only=True)
# model.fit(Xtrain,ytrain,batch_size=20, epochs=200,callbacks=model_checkpoint_callback,validation_data=(Xvalid,yvalid))
# loss,acc=model.evaluate(Xtest,ytest)
# print(acc)


In [25]:

M31_X,M31_y=np.array(dict_X["03"]),np.array(dict_y["03"])
F34_X,F34_y=np.array(dict_X["08"]),np.array(dict_y["08"])
F21_X,F21_y=np.array(dict_X["09"]),np.array(dict_y["09"])
M32_X,M32_y=np.array(dict_X["10"]),np.array(dict_y["10"])
M26_X,M26_y=np.array(dict_X["11"]),np.array(dict_y["11"])
M30_X,M30_y=np.array(dict_X["12"]),np.array(dict_y["12"])
F32_X,F32_y=np.array(dict_X["13"]),np.array(dict_y["13"])
F35_X,F35_y=np.array(dict_X["14"]),np.array(dict_y["14"])
M25_X,M25_y=np.array(dict_X["15"]),np.array(dict_y["15"])
F31_X,F31_y=np.array(dict_X["16"]),np.array(dict_y["16"])

In [26]:
import librosa

In [27]:
melspecs =[0 for i in range(len(path))]
Y = torch.zeros([len(path),7],dtype = int)
for i in range(len(path)):
    r, sr = librosa.load('EmoDB/wav/'+path[i], res_type='kaiser_fast')
    melspec = librosa.feature.melspectrogram(y=r, sr=sr,n_fft = 512, hop_length=160, win_length=320,n_mels=24)
    melspecs[i]=melspec
    Y[i][emotion_code[path[i][5]]]=1

In [28]:
testfeats_X=defaultdict(list)
testfeats_y=defaultdict(list)
for i in range(len(path)):
    testfeats_X[path[i][:2]].append(melspecs[i])
    testfeats_y[path[i][:2]].append(Y[i])

In [29]:
trainfeats_X=defaultdict(list)
trainfeats_y=defaultdict(list)
for j in range(len(path)):    
    for i in range(0,len(melspecs[j][1]),50):
        temp = melspecs[0][:,i:i+100]
        if len(temp[1])==100:
            trainfeats_X[path[j][:2]].append(melspecs[0][:,i:i+100])
            trainfeats_y[path[j][:2]].append(Y[j])

In [31]:
# for i in range()

In [32]:
train_set=[]
for j in range(10):    
    for i in range(0,len(trainfeats_X[dict[j]])):
        train_set.append((torch.tensor(trainfeats_X[dict[j]][i].transpose()),trainfeats_y[dict[j]][i]))

In [33]:
trainfeats_X[dict[j]][i].transpose().shape

(100, 24)

In [34]:
test_set = [] 
for j in range(10):
    for i in range(0,len(testfeats_X[dict[j]])):
        test_set.append((torch.tensor(testfeats_X[dict[j]][i].transpose()),testfeats_y[dict[j]][i]))

In [35]:
for i in range(10):
    print(torch.tensor(train_set[i][0]).shape)

torch.Size([100, 24])
torch.Size([100, 24])
torch.Size([100, 24])
torch.Size([100, 24])
torch.Size([100, 24])
torch.Size([100, 24])
torch.Size([100, 24])
torch.Size([100, 24])
torch.Size([100, 24])
torch.Size([100, 24])


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [36]:
for i in range(10):
    print(torch.tensor(test_set[i][0]).shape)

torch.Size([262, 24])
torch.Size([223, 24])
torch.Size([259, 24])
torch.Size([277, 24])
torch.Size([199, 24])
torch.Size([240, 24])
torch.Size([293, 24])
torch.Size([207, 24])
torch.Size([208, 24])
torch.Size([234, 24])


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [37]:
import random
random.shuffle(train_set)

In [38]:
def reshapex(x):  
    return torch.reshape(x,(1,x.shape[0],24))
def reshapey(x):  
    return torch.reshape(x,(1,7))

In [39]:
import torch  
import torch.nn as nn
# import torchvision.datasets as datasets
# import torchvision.transforms as transforms
import speechbrain as sb
from speechbrain.nnet.pooling import StatisticsPooling
from speechbrain.nnet.CNN import Conv1d
from speechbrain.nnet.linear import Linear
from speechbrain.nnet.activations import Softmax
from speechbrain.nnet.normalization import BatchNorm1d

class Xvector(torch.nn.Module):
    """This model extracts X-vectors for speaker recognition and diarization.
    Arguments
    ---------
    device : str
        Device used e.g. "cpu" or "cuda".
    activation : torch class
        A class for constructing the activation layers.
    tdnn_blocks : int
        Number of time-delay neural (TDNN) layers.
    tdnn_channels : list of ints
        Output channels for TDNN layer.
    tdnn_kernel_sizes : list of ints
        List of kernel sizes for each TDNN layer.
    tdnn_dilations : list of ints
        List of dilations for kernels in each TDNN layer.
    lin_neurons : int
        Number of neurons in linear layers.
    Example
    -------
    >>> compute_xvect = Xvector('cpu')
    >>> input_feats = torch.rand([5, 10, 40])
    >>> outputs = compute_xvect(input_feats)
    >>> outputs.shape
    torch.Size([5, 1, 512])
    """
    
    def __init__(
        self,
        device,
        activation,
        tdnn_blocks,
        tdnn_channels,
        tdnn_kernel_sizes,
        tdnn_dilations,
        lin_neurons,
        in_channels,
    ):
        
        super().__init__()
        self.blocks = nn.ModuleList()

        # TDNN layers
        for block_index in range(tdnn_blocks):
            out_channels = tdnn_channels[block_index]
            self.blocks.extend(
                [
                    Conv1d(
                        in_channels=in_channels,
                        out_channels=out_channels,
                        kernel_size=tdnn_kernel_sizes[block_index],
                        dilation=tdnn_dilations[block_index],
                    ),
                    activation(),
                    BatchNorm1d(input_size=out_channels),
                ]
            )
            in_channels = tdnn_channels[block_index]

        # Statistical pooling
        self.blocks.append(StatisticsPooling())

        # Final linear transformation
        self.blocks.append(
            Linear(
                input_size=out_channels * 2,
                n_neurons=lin_neurons,
                bias=True,
                combine_dims=False,
            )
        )
        self.blocks.append(Linear(7,input_size=lin_neurons))
#         self.blocks.append(Softmax())
        
        

    def forward(self, x, lens=None):
        """Returns the x-vectors.
        Arguments
        ---------
        x : torch.Tensor
        """

        for layer in self.blocks:
            try:
                x = layer(x, lengths=lens)
            except TypeError:
                x = layer(x)
        return x


In [40]:
compute_xvect = Xvector("cpu",torch.nn.LeakyReLU,5,[512, 512, 512, 512, 1500],[5, 3, 3, 1, 1],[1, 2, 3, 1, 1],512,24)

In [41]:
input_feats = [1,265,24]

In [42]:
input = torch.rand(input_feats)

In [43]:
output = compute_xvect(input)

In [44]:
output.shape

torch.Size([1, 1, 7])

In [45]:
import torch.optim as optim

In [46]:
valid_set = train_set[1840:]

In [76]:
def train_valid(train_set,valid_set):
    optimizer = optim.Adam(compute_xvect.parameters(), lr = 0.0001)
    criterion = nn.CrossEntropyLoss()
    epochs = 5
    min_validloss = np.inf
    for epoch in range(epochs):
        print(epoch)
        train_loss=0
        for data in train_set:    
            X, y = data
            compute_xvect.zero_grad()
            output = compute_xvect(reshapex(X))
            y=reshapey(y)
            loss=criterion(output[0], torch.argmax(y).view(-1))
            loss.backward()
            optimizer.step()
            train_loss+=loss.item()
        valid_loss = 0
        for data in valid_set:
            X, y = data
            output = compute_xvect(reshapex(X))
            y=reshapey(y)
            loss=criterion(output[0], torch.argmax(y).view(-1))
            valid_loss+=loss.item()

        if min_validloss>valid_loss:
            min_validloss = valid_loss    

In [77]:
# compute_xvect.eval()

In [78]:
def test(test_set):
    corr = 0
    total = 0
    with torch.no_grad():
        for data in test_set:
            X,y=data
            output = compute_xvect(reshapex(X))
            for idx,i in enumerate(output):
                if torch.argmax(i)==y[idx]:
                    corr+=1
                total+=1
    return corr*100/total
    print("Accuracy = "+str(corr*100/total))

In [79]:
temp = 9
acc = 0
accuracies = [0]*10
for k in range(10):
    if temp == 0:
        valid = 9
    else:
        valid = temp-1
    train_set = []
    test_set  = []
    valid_set = []
    print(valid,temp)
    for j in range(10):
        if j not in [temp,valid]:
            for i in range(0,len(trainfeats_X[dict[j]])):
                train_set.append((torch.tensor(trainfeats_X[dict[j]][i].transpose()),trainfeats_y[dict[j]][i]))
    for i in range(0,len(trainfeats_X[dict[valid]])):
        valid_set.append((torch.tensor(trainfeats_X[dict[valid]][i].transpose()),trainfeats_y[dict[valid]][i]))
    for i in range(0,len(testfeats_X[dict[temp]])):
        test_set.append((torch.tensor(testfeats_X[dict[temp]][i].transpose()),testfeats_y[dict[temp]][i]))
    train_valid(train_set,valid_set)
    temp_acc = test(test_set)
    accuracies[temp]=temp_acc
    print(f'Accuracy for iteration: {temp_acc}')
    acc += temp_acc
    temp-=1

8 9
0
1
2
3
4
Accuracy for iteration: 80.28169014084507
7 8
0
1
2
3
4
Accuracy for iteration: 76.78571428571429
6 7
0
1
2
3
4
Accuracy for iteration: 76.81159420289855
5 6
0
1
2
3
4
Accuracy for iteration: 80.32786885245902
4 5
0
1
2
3
4
Accuracy for iteration: 65.71428571428571
3 4
0
1
2
3
4
Accuracy for iteration: 80.0
2 3
0
1
2
3
4
Accuracy for iteration: 73.6842105263158
1 2
0
1
2
3
4
Accuracy for iteration: 69.76744186046511
0 1
0
1
2
3
4
Accuracy for iteration: 79.3103448275862
9 0
0
1
2
3
4
Accuracy for iteration: 71.42857142857143


In [80]:
accuracies

[71.42857142857143,
 79.3103448275862,
 69.76744186046511,
 73.6842105263158,
 80.0,
 65.71428571428571,
 80.32786885245902,
 76.81159420289855,
 76.78571428571429,
 80.28169014084507]

In [82]:
sum(accuracies)/10

75.41117218391412